In [1]:
# Install and load required packages (if not already installed)
install.packages(c("googledrive", "readr","knitr","ecodist","kableExtra","magrittr","DT","magick","grid"))


Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘grid’ is a base package, and should not be updated”
also installing the dependencies ‘lazyeval’, ‘igraph’, ‘svglite’, ‘crosstalk’


Warning message in install.packages(c("googledrive", "readr", "knitr", "ecodist", :
“installation of package ‘magick’ had non-zero exit status”


In [2]:
library(googledrive)
library(readr)
library(knitr)
library(ecodist)
library(kableExtra)
library(magrittr)
library(DT)
library(grid)
drive_auth()

Is it OK to cache OAuth access credentials in the folder ~/.cache/gargle
between R sessions?
1: Yes
2: No


Selection: 1


Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=27b2b217973051b2e3a7e493ba907fef&access_type=offline&prompt=consent



Enter authorization code: eyJjb2RlIjoiNC8wQWN2RE1yQkEyb0dFRjZfUVd5dFhwajFuRGRsR296T0lzbzB2XzFMODFhekY0TXBVSUJvamVjbmVOY2Fob2I3Q1F2ZXdaUSIsInN0YXRlIjoiMjdiMmIyMTc5NzMwNTFiMmUzYTdlNDkzYmE5MDdmZWYifQ==


In [3]:
# Read all CSV files from Google Drive
v1 <- read.csv("https://drive.google.com/uc?export=download&id=1-AftjsRKwi8IWId10Ile5xHa22Chf51q")
v2 <- read.csv("https://drive.google.com/uc?export=download&id=1-T2LJhl9-b7_nLpJ-QHUUpnf_1arsCPI")
v3 <- read.csv("https://drive.google.com/uc?export=download&id=1-UKbiS_eu63c20fzwA2c-3Sf6KX3G4BK")
v3a <- read.csv("https://drive.google.com/uc?export=download&id=1-Vs1tsNuXnviVs_4VkO3ExV6nUa3MUvI")
v3b <- read.csv("https://drive.google.com/uc?export=download&id=1-W6c9k5fqgYfTC3CAevEtUx2o4e6cFhD")
v4 <- read.csv("https://drive.google.com/uc?export=download&id=1-gNfCQYVBiexOJESQQz73t4IV8lgpx9U")
hmax_c1 <- read.csv("https://drive.google.com/uc?export=download&id=104aNbBRnKC_Xg615Sgnk70TUUni_wy-I")
latocc <- read.csv("https://drive.google.com/uc?export=download&id=1-jc6pOFmB0RnqOnshG-eahCn93EHOhcp")
anim_inanim <- read.csv("https://drive.google.com/uc?export=download&id=101N1Ma0R1HvGV6DJyC8XrdgIbfHeZuyr")




In [4]:
hmax_s1 <- read.csv("https://drive.google.com/uc?export=download&id=1-2GCpqi_xb7XBdBSQfL0dLNCspVIytjr")


In [5]:
# Print the shape of each data frame
cat("v1 dims:",dim(v1),"\n")
cat("v2 dimensions:", dim(v2), "\n")
cat("v3 dimensions:", dim(v3), "\n")
cat("v3a dimensions:", dim(v3a), "\n")
cat("v3b dimensions:", dim(v3b), "\n")
cat("v4 dimensions:", dim(v4), "\n")
cat("hmax dimensions:", dim(hmax_c1), "\n")
cat("latocc dimensions:", dim(latocc), "\n")
cat("anim_inanim dimensions:", dim(anim_inanim), "\n")

v1 dims: 1750 1750 
v2 dimensions: 1750 1750 
v3 dimensions: 1750 1750 
v3a dimensions: 1750 1750 
v3b dimensions: 1750 1750 
v4 dimensions: 1750 1750 
hmax dimensions: 1750 1750 
latocc dimensions: 1750 1750 
anim_inanim dimensions: 1750 1750 


In [6]:
library(knitr)
library(ecodist)

In [9]:
# Create lists for target and predictor tables
targetTables <- list(v1, v2, v3, v3a, v3b, v4, latocc)
predtables <- list(hmax_c1, anim_inanim)

# Initialize an empty list to store result data frames
result_list <- list()

# Note: The initialization of result_list is repeated, we can remove one instance

# Loop over the targetTables
for (k in seq_along(targetTables)) {
  # Construct the formula dynamically
  formula <- paste0("as.dist(targetTables[[", k, "]]) ~ ",
                    paste(paste0("as.dist(predtables[[", seq_along(predtables), "]])"), collapse = " + "))

  # Perform MRM using all predictor tables
  mrm_result <- MRM(formula = formula, nperm = 2500)

  # Convert the result to a data frame
  result_df <- data.frame(
    Target = rep(paste0("v", k), nrow(mrm_result$coef)),  # This might need adjustment for 'latocc'
    Predictor = c("Intercept", sub("_rdm.csv", "", c("hmax", "anim_inanim"))),
    Coefficient = mrm_result$coef[,1],
    P_Value_Coef = mrm_result$coef[,2],
    R_Squared = rep(mrm_result$r.squared[1], nrow(mrm_result$coef)),
    P_Value_R_Squared = rep(mrm_result$r.squared[2], nrow(mrm_result$coef)),
    F_Statistic = rep(mrm_result$F.test[1], nrow(mrm_result$coef)),
    P_Value_F_Statistic = rep(mrm_result$F.test[2], nrow(mrm_result$coef))
  )

  print(kable(result_df, format = "markdown", caption = paste0("Results for target table ", k)))

  result_list[[k]] <- result_df
}

all_results <- do.call(rbind, result_list)



Table: Results for target table 1

|                         |Target |Predictor   | Coefficient| P_Value_Coef| R_Squared| P_Value_R_Squared| F_Statistic| P_Value_F_Statistic|
|:------------------------|:------|:-----------|-----------:|------------:|---------:|-----------------:|-----------:|-------------------:|
|Int                      |v1     |Intercept   |  49.6727295|       0.9324| 0.0005144|            0.1312|    393.7921|              0.1312|
|as.dist(predtables[[1]]) |v1     |hmax        |   0.0764936|       0.1312| 0.0005144|            0.1312|    393.7921|              0.1312|
|as.dist(predtables[[2]]) |v1     |anim_inanim |   0.0077772|       0.0712| 0.0005144|            0.1312|    393.7921|              0.1312|


Table: Results for target table 2

|                         |Target |Predictor   | Coefficient| P_Value_Coef| R_Squared| P_Value_R_Squared| F_Statistic| P_Value_F_Statistic|
|:------------------------|:------|:-----------|-----------:|------------:|---------:|